In [1]:
import os,inspect
import os.path
from pathlib import Path

In [2]:
tr_dir_fake = os.listdir("train/training_fake/")
tr_dir_real = os.listdir("train/training_real/")
#tst_dir = os.listdir("test/")

In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame(columns=['class','dir','image_file'])
#test_df = pd.DataFrame(columns=['class','dir','image_file'])

In [4]:
for name in tr_dir_fake:
    im_dir = "train/training_fake"
    im_class = 'fake'
    df.loc[df.shape[0]] = [im_class, im_dir, "train/training_fake/"+name]

for name in tr_dir_real:
    im_dir = "train/training_real"
    im_class = 'real'
    df.loc[df.shape[0]] = [im_class, im_dir, "train/training_real/"+name]

In [5]:
df

,class,dir,image_file
0,fake,train/training_fake,train/training_fake/0.jpg
1,fake,train/training_fake,train/training_fake/1.jpg
2,fake,train/training_fake,train/training_fake/10.jpg
3,fake,train/training_fake,train/training_fake/100.jpg
4,fake,train/training_fake,train/training_fake/101.jpg
...,...,...,...
1704,real,train/training_real,train/training_real/995.jpg
1705,real,train/training_real,train/training_real/996.jpg
1706,real,train/training_real,train/training_real/997.jpg
1707,real,train/training_real,train/training_real/998.jpg


In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
train_df

,class,dir,image_file
590,fake,train/training_fake,train/training_fake/63.jpg
1160,real,train/training_real,train/training_real/1400.jpg
420,fake,train/training_fake,train/training_fake/477.jpg
425,fake,train/training_fake,train/training_fake/481.jpg
859,real,train/training_real,train/training_real/1099.jpg
...,...,...,...
1130,real,train/training_real,train/training_real/1370.jpg
1294,real,train/training_real,train/training_real/1534.jpg
860,real,train/training_real,train/training_real/1100.jpg
1459,real,train/training_real,train/training_real/1699.jpg


In [8]:
test_df

,class,dir,image_file
567,fake,train/training_fake,train/training_fake/609.jpg
1326,real,train/training_real,train/training_real/1566.jpg
1061,real,train/training_real,train/training_real/1301.jpg
115,fake,train/training_fake,train/training_fake/201.jpg
453,fake,train/training_fake,train/training_fake/506.jpg
...,...,...,...
1205,real,train/training_real,train/training_real/1445.jpg
1111,real,train/training_real,train/training_real/1351.jpg
128,fake,train/training_fake,train/training_fake/213.jpg
405,fake,train/training_fake,train/training_fake/463.jpg


In [9]:
#for name in tst_dir:
#    im_dir = "test"
#    im_class = 'fake'
#    test_df.loc[test_df.shape[0]] = [im_class, im_dir, "test/"+name]

In [10]:
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv', index=False)

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [12]:
def batch_generator(df, batchsize=32):   
    
    while True:       
                
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0], batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img(df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class        
        
        yield batch_images, batch_labels          

In [13]:
img_size = 32
img_depth = 3

In [14]:
class_names = train_df['class'].unique().tolist()
class_names

['fake', 'real']

In [15]:
import tensorflow as tf

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add Layers
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

In [16]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1605760   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [18]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize)

In [ ]:
model.fit_generator(train_generator, 
                          epochs=10,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

C:\Users\subha\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 7/21 [=========>....................] - ETA: 15s - loss: 0.7051 - accuracy: 0.5392

In [ ]:
tst_dir = os.listdir("test/")
test_image = []
for i in tst_dir:
    test_image.append('test/'+i)
test_image

In [ ]:
test_model_pred = []
for j in test_image:
    inp_image = tf.keras.preprocessing.image.load_img(j)
    model_input = inp_image.resize((32,32))
    model_input = tf.keras.preprocessing.image.img_to_array(model_input)
    model_input = np.expand_dims(model_input, axis=0)
    pred = model.predict(model_input)
    test_model_pred.append(np.argmax(pred))

In [ ]:
test_image1 = [k.strip('test/') for k in test_image]
test_image2 = [k.strip('.jpg') for k in test_image1]

In [ ]:
submission_df = pd.DataFrame(list(zip(test_image2, test_model_pred)),columns =['file_id', 'label'])

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("Final_submission.csv")